In [ ]:
%pip install lmfit
%pip install xlrd

In [ ]:
%pip install -q ipympl

In [ ]:
import scipy
import matplotlib.pyplot as plt
import lmfit
import io
import numpy as np
import pandas as pd

In [ ]:
%matplotlib inline

# Read rheology data

In [ ]:
Data_file=pd.ExcelFile('Flow_curve_example.xls')
Data_file.sheet_names

In [ ]:
data=Data_file.parse('Flow sweep - 2',skiprows=[0,2])
data.head()

# define model : Three component model

$$\sigma=\sigma_y+\sigma_y\cdot(\dot\gamma/\dot\gamma_c)^{0.5}+\eta_{bg}\cdot\dot\gamma$$

In [ ]:
def TC(x, ystress=1.0, eta_bg=0.1, gammadot_crit=0.1):
    """Three component model
    Note:
    .. math::
       \sigma=\sigma_y+\sigma_y\cdot(\dot\gamma/\dot\gamma_c)^{0.5}+\eta_{bg}\cdot\dot\gamma
    Args:
        ystress: yield stress [Pa]
        eta_bg : Background viscosity [Pa s]
        gammadot_crit : Critical shear rate [1/s]
    Returns:
        stress : Shear Stress, [Pa]
    """
    return ystress + ystress * (x / gammadot_crit) ** 0.5 + eta_bg * x

TC_model = lmfit.Model(TC, prefix="TC_")

# set parameters for model class
TC_model.set_param_hint("ystress", min=0)
TC_model.set_param_hint("eta_bg", min=0, vary=True)
TC_model.set_param_hint("gammadot_crit", min=0)


def HB(x, ystress=1.0, K=1.0, n=0.5):
    """Hershel-Bulkley Model
    Note:
    .. math::
       \sigma= \sigma_y + K \cdot \dot\gamma^n
    Args:
        ystress: yield stress [Pa]
        K : Consistency index [Pa s^n]
        n : Shear thinning index []
    Returns:
        stress : Shear Stress, [Pa]
    """
    return ystress + K * x ** n


HB_model = lmfit.Model(HB, prefix="HB_")

HB_model.set_param_hint("ystress", min=0)
HB_model.set_param_hint("K", min=0, vary=True)
HB_model.set_param_hint("n", min=0.0, max=1, vary=True)

In [ ]:
res_fit_TC=TC_model.fit(data['Stress'],x=data['Shear rate'], weights=1/data['Stress'])
res_fit_HB=HB_model.fit(data['Stress'],x=data['Shear rate'], weights=1/data['Stress'])

In [ ]:
plt.plot(data['Shear rate'],res_fit_TC.best_fit)
plt.plot(data['Shear rate'],res_fit_HB.best_fit)

plt.plot(data['Shear rate'], data['Stress'], 'o')
plt.xlabel('Shear rate 1/s')
plt.ylabel('Stress [Pa]')
plt.xscale('log')
plt.yscale('log')

In [ ]:
import pybroom as pb


In [ ]:
tidy_long=pb.tidy({'TC_example':res_fit_TC, 'HB_example':res_fit_HB})
tidy_long

In [ ]:
tidy_wide=tidy_long.pivot(index='key',columns='name',values='value')
tidy_wide.reset_index()

In [ ]:
tidy_wide.reset_index().to_excel('results.xlsx')